In [22]:
from yahoo_oauth import OAuth2
import os
import json
import league, game, team
import yhandler
import json
import objectpath
import pandas as pd
from pandas.io.json import json_normalize

In [23]:
# get our session context ("sc")
oauth = OAuth2(None, None, from_file='oauth2.json')

[2019-08-25 12:18:53,459 DEBUG] [yahoo_oauth.yahoo_oauth.__init__] Checking 
[2019-08-25 12:18:53,518 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 86427.2998650074
[2019-08-25 12:18:53,519 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN HAS EXPIRED
[2019-08-25 12:18:53,520 DEBUG] [yahoo_oauth.yahoo_oauth.refresh_access_token] REFRESHING TOKEN


In [24]:
# prepare functions
def get_json_dump(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    return json.dumps(jresp)
def get_json(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    return jresp
def get_json_t(url):
    YAHOO_ENDPOINT = 'https://fantasysports.yahooapis.com/fantasy/v2'
    response = oauth.session.get("{}/{}".format(YAHOO_ENDPOINT, url),params={'format': 'json'})
    jresp = response.json()
    t = objectpath.Tree(jresp)
    return t

https://developer.yahoo.com/fantasysports/guide/
The format for requesting a Resource will typically look like:
https://fantasysports.yahooapis.com/fantasy/v2/{resource}/{resource_key}

While the format for requesting a Collection will typically look like:
https://fantasysports.yahooapis.com/fantasy/v2/{collection};{resource}_keys={resource_key1},{resource_key2}

Collection:
/fantasy/v2/;player_keys=,{player_key2}
/fantasy/v2/;use_login=1

If you ever need to branch off other sub-resources outside of your main resource chain, you can use the out parameter, which will let you specify one level of extra sub-resources to pull in. At the moment, you cannot pass any parameters along to these out sub-resources, aside from any data that might get passed by default. This typically means that you can’t chain other resources off of sub-resources specified by the out parameter.
As an example, if you wanted to view a league’s settings along with two teams in particular in a league, you might construct a URI like:
/fantasy/v2//;out=settings/;team_keys=,{team_key2}

Parameters¶
Parameters can be provided to Resources and Collections as semicolon-delimited key-value pairs. These should be placed after the Resource or Collection name in the URI; in the case of entry-point Resources like Games, Leagues, Teams, and Players, the parameters belong after the resource_key.

/fantasy/v2/{resource}/{resource_key};{key}={value};{key}={value}/{collection};{key}={value};{key}={value}/{collection};{key}={value}/{resource};{key}={value}

URIs¶
Game Resource:
https://fantasysports.yahooapis.com/fantasy/v2/game/
Any sub-resource under a game is extracted using a URI like:
https://fantasysports.yahooapis.com/fantasy/v2/game//
Multiple sub-resources can be extracted from game in the same URI using a format like:
https://fantasysports.yahooapis.com/fantasy/v2/game/;out=,{sub_resource_2}
https://fantasysports.yahooapis.com/fantasy/v2/game/nfl

League:
https://fantasysports.yahooapis.com/fantasy/v2/league/
Any sub-resource under a league is extracted using a URI like:
https://fantasysports.yahooapis.com/fantasy/v2/league//
Multiple sub-resources can be extracted from league in the same URI using a format like:
https://fantasysports.yahooapis.com/fantasy/v2/league/;out=,{sub_resource_2}
Examples:
https://fantasysports.yahooapis.com/fantasy/v2/league/223.l.431
https://fantasysports.yahooapis.com/fantasy/v2/league/223.l.431/settings
https://fantasysports.yahooapis.com/fantasy/v2/league/223.l.431/standings
https://fantasysports.yahooapis.com/fantasy/v2/league/223.l.431/scoreboard


Team Resource
https://fantasysports.yahooapis.com/fantasy/v2/team/
Any sub-resource under a team is extracted using a URI like:
https://fantasysports.yahooapis.com/fantasy/v2/team//
Multiple sub-resources can be extracted from team in the same URI using a format like:
https://fantasysports.yahooapis.com/fantasy/v2/team/;out=,{sub_resource_2}
Examples:
https://fantasysports.yahooapis.com/fantasy/v2/team/223.l.431.t.1
https://fantasysports.yahooapis.com/fantasy/v2/team/223.l.431.t.1/matchups;weeks=1,5 - team’s matchups for weeks 1 and 5 in a NFL H2H league
https://fantasysports.yahooapis.com/fantasy/v2/team/223.l.431.t.1/stats;type=season - team’s season stats in a NFL H2H league

Roster Resource
https://fantasysports.yahooapis.com/fantasy/v2/team//roster
Any sub-resource under a roster is extracted using a URI like:
https://fantasysports.yahooapis.com/fantasy/v2/team//roster/
For NFL, you can specify a week parameter to retrieve a specific week – otherwise it will default to the current roster
https://fantasysports.yahooapis.com/fantasy/v2/team//roster;week=10
For MLB, NHL, or NBA, you can specify a date parameter to retrieve a specific date – otherwise it will default to today’s roster.
https://fantasysports.yahooapis.com/fantasy/v2/team//roster;date=2011-05-01
Default sub-resource: PLAYERS

Examples: https://fantasysports.yahooapis.com/fantasy/v2/team/253.l.102614.t.10/roster/players
You can do a "put" request to move players. See https://developer.yahoo.com/fantasysports/guide/#put

Teams Collection
Any sub-resource valid for a team is a valid sub-resource under the teams collection.
Any sub-resource for a collection of teams is extracted using a URI like:
/teams/{sub_resource}
OR
/teams;team_keys={team_key1},{team_key2}/{sub_resource}
Multiple sub-resources can be extracted from teams in the same URI using a format like:
/teams;out={sub_resource_1},{sub_resource_2}
OR
/teams;team_keys={team_key1},{team_key2};out={sub_resource_1},{sub_resource_2}


Players Resource
https://fantasysports.yahooapis.com/fantasy/v2/player/
Any sub-resource under a player is extracted using a URI like:
https://fantasysports.yahooapis.com/fantasy/v2/player//
Multiple sub-resources can be extracted from player in the same URI using a format like:
https://fantasysports.yahooapis.com/fantasy/v2/player/;out=,{sub_resource_2}

https://fantasysports.yahooapis.com/fantasy/v2/league/223.l.431/players;player_keys=223.p.5479 - Player in a NFL league context
https://fantasysports.yahooapis.com/fantasy/v2/league/223.l.431/players;player_keys=223.p.5479/stats - Player season stats in a NFL league context


Players Collection
Any sub-resource valid for a player is a valid sub-resource under the players collection.
Any sub-resource for a collection of players is extracted using a URI like:
/players/{sub_resource}
OR
/players;player_keys={player_key1},{player_key2}/{sub_resource}
Multiple sub-resources can be extracted from players in the same URI using a format like:
/players;out={sub_resource_1},{sub_resource_2}
OR
/players;player_keys={player_key1},{player_key2};out={sub_resource_1},{sub_resource_2}

Transactions...
https://developer.yahoo.com/fantasysports/guide/#post


In [25]:
#league ids: kffl_league_id = '390.l.707700', joesleagueid = '390.l.974698', jwold = '371.l.252880', jwnew: '380.l.770342'
# gm = game.Game(oauth, 'nfl')
# print(gm)
# https://fantasysports.yahooapis.com/fantasy/v2/league/ 
gm = game.Game(oauth, 'nfl')

#use package to get league_id
league_ids = gm.league_ids()
print(league_ids)
test_league_id = league_ids[2]
print(test_league_id)


['371.l.252880', '380.l.770342', '390.l.974698']
390.l.974698


In [26]:
#get league
lg = gm.to_league(test_league_id)
print(f"scoring categories: {lg.stat_categories()}")
print(f"standings:{lg.standings()}")
print(f"teams:{lg.teams()}")
print(f"current user team:{lg.team_key()}")
print(f"current week:{lg.current_week()}")

scoring categories: [{'display_name': 'Pass Yds', 'position_type': 'O'}, {'display_name': 'Pass TD', 'position_type': 'O'}, {'display_name': 'Int', 'position_type': 'O'}, {'display_name': 'Rush Yds', 'position_type': 'O'}, {'display_name': 'Rush TD', 'position_type': 'O'}, {'display_name': 'Rec', 'position_type': 'O'}, {'display_name': 'Rec Yds', 'position_type': 'O'}, {'display_name': 'Rec TD', 'position_type': 'O'}, {'display_name': 'Ret TD', 'position_type': 'O'}, {'display_name': '2-PT', 'position_type': 'O'}, {'display_name': 'Fum Lost', 'position_type': 'O'}, {'display_name': 'Fum Ret TD', 'position_type': 'O'}, {'display_name': 'FG 0-19', 'position_type': 'K'}, {'display_name': 'FG 20-29', 'position_type': 'K'}, {'display_name': 'FG 30-39', 'position_type': 'K'}, {'display_name': 'FG 40-49', 'position_type': 'K'}, {'display_name': 'FG 50+', 'position_type': 'K'}, {'display_name': 'PAT Made', 'position_type': 'K'}, {'display_name': 'Sack', 'position_type': 'DT'}, {'display_name':

In [27]:
# get league teams:
teams = lg.teams()
teams

[{'name': "JCommish's Runts", 'team_key': '390.l.974698.t.1'},
 {'name': 'AHFanGooglers', 'team_key': '390.l.974698.t.2'},
 {'name': "JoeY!'s Scrubs", 'team_key': '390.l.974698.t.3'},
 {'name': "JGH's Slackers", 'team_key': '390.l.974698.t.4'}]

In [ ]:
#testteams
# [{'name': "JCommish's Runts", 'team_key': '390.l.974698.t.1'},
#  {'name': 'AHFanGooglers', 'team_key': '390.l.974698.t.2'},
#  {'name': "JoeY!'s Scrubs", 'team_key': '390.l.974698.t.3'},
#  {'name': "JGH's Slackers", 'team_key': '390.l.974698.t.4'}]
rosterurl = "/team/390.l.974698.t.1/roster"
test = get_json_dump(rosterurl)
test

In [45]:
rostert = get_json_t(rosterurl)
jfilter = rostert.execute('$..(player)')

for row in jfilter:
    print(row)
    print("-----------------------------------------")
    print("---------------nextrow-------------------")
    print("-----------------------------------------")





{'player': [[{'player_key': '390.p.30123'}, {'player_id': '30123'}, {'name': {'full': 'Patrick Mahomes', 'first': 'Patrick', 'last': 'Mahomes', 'ascii_first': 'Patrick', 'ascii_last': 'Mahomes'}}, {'editorial_player_key': 'nfl.p.30123'}, {'editorial_team_key': 'nfl.t.12'}, {'editorial_team_full_name': 'Kansas City Chiefs'}, {'editorial_team_abbr': 'KC'}, {'bye_weeks': {'week': '12'}}, {'uniform_number': '15'}, {'display_position': 'QB'}, {'headshot': {'url': 'https://s.yimg.com/iu/api/res/1.2/mI5OqDbobMP0SIlpOv8XCQ--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nfl_cutout/players_l/08152019/30123.png', 'size': 'small'}, 'image_url': 'https://s.yimg.com/iu/api/res/1.2/mI5OqDbobMP0SIlpOv8XCQ--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nfl_cutout/players_l/08152019/30123.png'}, {'is_undroppable': '1'}, {'position_typ